In [1]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re 
import pandas as pd

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('Tinder_Data_v2.csv')

In [3]:
df.head()

,_id,sum_app_opens,no_of_days_x,nrOfConversations,longestConversation,longestConversationInDays,averageConversationLength,averageConversationLengthInDays,medianConversationLength,medianConversationLengthInDays,nrOfOneMessageConversations,percentOfOneMessageConversations,nrOfGhostingsAfterInitialMessage,no_of_days_y,no_of_matches,no_of_msgs_sent,no_of_msgs_received,swipe_likes,swipe_passes,birthDate,ageFilterMin,ageFilterMax,cityName,country,createDate,education,gender,interestedIn,genderFilter,instagram,spotify,jobTitle,educationLevel
0,00b74e27ad1cbb2ded8e907fcc49eaaf,6839,477,739,133,683.557442,8.560217,10.236620,3,0.081134,226,30.581867,66,477,3408,3360,3261,23341,24229,1976-01-01T00:00:00.000Z,21,35,Trondheim,Norway,2016-01-01T09:30:07.551Z,Has high school and/or college education,M,F,F,False,False,NaN,Has high school and/or college education
1,024610702baf540af5637873cd1534e9,26280,1197,464,36,622.466528,4.495690,3.973980,2,0.000069,56,12.068966,16,1197,888,2088,1509,45068,49559,1997-07-04T00:00:00.000Z,18,29,Richmond,VA,2016-07-12T02:26:46.774Z,Has high school and/or college education,M,F,F,False,False,NaN,Has high school and/or college education
2,0a5e3dd8489fe67485ddb7d6adb26ebd,3196,249,303,33,295.283414,4.254125,3.465479,2,0.033715,106,34.983498,17,249,548,1291,1152,6679,5500,1998-02-06T00:00:00.000Z,20,24,NaN,NaN,2019-07-01T19:17:54.560Z,Has no high school or college education,M,F,F,False,False,NaN,Has no high school or college education
3,048dd37565ad9cbc24c163ffedffbf58,2077,158,47,28,130.034410,7.893617,5.878985,6,0.643727,3,6.382979,0,158,94,371,224,6724,7899,1998-02-24T00:00:00.000Z,20,25,Edmonton,Alberta,2019-09-25T03:28:20.920Z,Has no high school or college education,M,F,F,False,False,NaN,Has no high school or college education
4,0eb998fdde77f9c123c07eace18a5cc1,11946,715,809,444,198.390972,6.824475,1.924996,3,0.091771,296,36.588381,13,715,1905,5527,6052,60169,96673,1996-11-10T00:00:00.000Z,18,27,NaN,NaN,2017-11-17T23:30:37.231Z,Has no high school or college education,M,F,F,True,False,Research Assistant,Has no high school or college education


In [4]:
# Check if two columns are the same
are_columns_equal = df['education'] == df['educationLevel']

# Print the result
print("Are the two columns equal?", are_columns_equal.all())


Are the two columns equal? True


In [5]:
df = df.drop(columns=['educationLevel'])

In [6]:
# Round the values in the 'longestConversationInDays' column
df['longestConversationInDays'] = np.round(df['longestConversationInDays']).astype(int)
df['averageConversationLength'] = df['averageConversationLength'].round(2)
df['averageConversationLengthInDays'] = df['averageConversationLengthInDays'].round(2)

In [7]:
# Check if two columns are the same
are_columns_equal = df['no_of_days_x'] == df['no_of_days_y']

# Print the result
print("Are the two columns equal?", are_columns_equal.all())

Are the two columns equal? False


In [8]:
df = df.drop(columns=['no_of_days_y'])

In [9]:
df = df.rename(columns={'no_of_days_x': 'no_of_days'})


In [10]:
df['percentOfOneMessageConversations'] = df['percentOfOneMessageConversations'].round(2)

In [11]:
# Convert the 'birthDate' column to a datetime object
df['birthDate'] = pd.to_datetime(df['birthDate'])

# Format the 'birthDate' column to the desired format (mm-dd-year)
df['birthDate'] = df['birthDate'].dt.strftime('%m-%d-%Y')


In [12]:
# Convert the 'birthDate' column to a datetime object
df['createDate'] = pd.to_datetime(df['createDate'])

# Format the 'birthDate' column to the desired format (mm-dd-year)
df['createDate'] = df['createDate'].dt.strftime('%m-%d-%Y')


Feature Engineering Idea - derive user's current age by using createdate - birthdate

In [13]:
df.head()

,_id,sum_app_opens,no_of_days,nrOfConversations,longestConversation,longestConversationInDays,averageConversationLength,averageConversationLengthInDays,medianConversationLength,medianConversationLengthInDays,nrOfOneMessageConversations,percentOfOneMessageConversations,nrOfGhostingsAfterInitialMessage,no_of_matches,no_of_msgs_sent,no_of_msgs_received,swipe_likes,swipe_passes,birthDate,ageFilterMin,ageFilterMax,cityName,country,createDate,education,gender,interestedIn,genderFilter,instagram,spotify,jobTitle
0,00b74e27ad1cbb2ded8e907fcc49eaaf,6839,477,739,133,684,8.56,10.24,3,0.081134,226,30.58,66,3408,3360,3261,23341,24229,01-01-1976,21,35,Trondheim,Norway,01-01-2016,Has high school and/or college education,M,F,F,False,False,NaN
1,024610702baf540af5637873cd1534e9,26280,1197,464,36,622,4.50,3.97,2,0.000069,56,12.07,16,888,2088,1509,45068,49559,07-04-1997,18,29,Richmond,VA,07-12-2016,Has high school and/or college education,M,F,F,False,False,NaN
2,0a5e3dd8489fe67485ddb7d6adb26ebd,3196,249,303,33,295,4.25,3.47,2,0.033715,106,34.98,17,548,1291,1152,6679,5500,02-06-1998,20,24,NaN,NaN,07-01-2019,Has no high school or college education,M,F,F,False,False,NaN
3,048dd37565ad9cbc24c163ffedffbf58,2077,158,47,28,130,7.89,5.88,6,0.643727,3,6.38,0,94,371,224,6724,7899,02-24-1998,20,25,Edmonton,Alberta,09-25-2019,Has no high school or college education,M,F,F,False,False,NaN
4,0eb998fdde77f9c123c07eace18a5cc1,11946,715,809,444,198,6.82,1.92,3,0.091771,296,36.59,13,1905,5527,6052,60169,96673,11-10-1996,18,27,NaN,NaN,11-17-2017,Has no high school or college education,M,F,F,True,False,Research Assistant


In [14]:
# Assuming 'column_name' is the name of the column for which you want to print unique values
unique_values = df['jobTitle'].unique()

# Print the unique values
#print(unique_values)


In [15]:
# Assuming 'column_name' is the name of the column for which you want to print unique values
unique_values = df['gender'].unique()

# Print the unique values
print(unique_values)


['M' 'F' nan]


In [16]:
# Assuming 'column_name' is the name of the column for which you want to count unique values
unique_value_counts = df['gender'].value_counts()

# Print the counts of unique values
print(unique_value_counts)


M    1058
F     150
Name: gender, dtype: int64


**Substituting Missing Gender, InterestedIn and genderFilter**

In [17]:
# Find the mode of the 'gender' column
gender_mode = df['gender'].mode().iloc[0]

# Replace null values in the 'gender' column with the mode
df['gender'].fillna(gender_mode, inplace=True)


In [18]:
# Find the mode of the 'gender' column
interested_in_mode = df['interestedIn'].mode().iloc[0]

# Replace null values in the 'gender' column with the mode
df['interestedIn'].fillna(interested_in_mode, inplace=True)

In [19]:
# Find the mode of the 'gender' column
genderFilter_mode = df['genderFilter'].mode().iloc[0]

# Replace null values in the 'gender' column with the mode
df['genderFilter'].fillna(genderFilter_mode, inplace=True)

In [20]:
# Check if two columns are the same
are_columns_equal = df['interestedIn'] == df['genderFilter']

# Print the result
print("Are the two columns equal?", are_columns_equal.all())

Are the two columns equal? True


In [21]:
df = df.drop(columns=['genderFilter'])

In [22]:
# Replace missing values in 'cityName' and 'country' columns with 'unknown'
df['cityName'].fillna('unknown', inplace=True)
df['country'].fillna('unknown', inplace=True)
df['jobTitle'].fillna('unknown', inplace=True)


In [23]:
nan_counts = df.isna().sum()

# Print the counts of NaN values for each column
print(nan_counts)


_id                                 0
sum_app_opens                       0
no_of_days                          0
nrOfConversations                   0
longestConversation                 0
longestConversationInDays           0
averageConversationLength           0
averageConversationLengthInDays     0
medianConversationLength            0
medianConversationLengthInDays      0
nrOfOneMessageConversations         0
percentOfOneMessageConversations    0
nrOfGhostingsAfterInitialMessage    0
no_of_matches                       0
no_of_msgs_sent                     0
no_of_msgs_received                 0
swipe_likes                         0
swipe_passes                        0
birthDate                           0
ageFilterMin                        0
ageFilterMax                        0
cityName                            0
country                             0
createDate                          0
education                           0
gender                              0
interestedIn

In [24]:
# Assuming 'column_name' is the name of the column for which you want to count unique values
unique_value_counts = df['country'].value_counts()

# Print the counts of unique values
print(unique_value_counts)

unknown              480
CA                    27
England               20
Germany               19
United Kingdom        18
                    ... 
Western Australia      1
Île-de-France          1
País Vasco             1
Łódzkie                1
South Africa           1
Name: country, Length: 243, dtype: int64


In [25]:
df['birthDate'] = pd.to_datetime(df['birthDate'])
df['createDate'] = pd.to_datetime(df['createDate'])
# Calculate the difference in years and create the 'user_age' column
df['user_age'] = (df['createDate'] - df['birthDate']).dt.days // 365

In [26]:
df.head()

,_id,sum_app_opens,no_of_days,nrOfConversations,longestConversation,longestConversationInDays,averageConversationLength,averageConversationLengthInDays,medianConversationLength,medianConversationLengthInDays,nrOfOneMessageConversations,percentOfOneMessageConversations,nrOfGhostingsAfterInitialMessage,no_of_matches,no_of_msgs_sent,no_of_msgs_received,swipe_likes,swipe_passes,birthDate,ageFilterMin,ageFilterMax,cityName,country,createDate,education,gender,interestedIn,instagram,spotify,jobTitle,user_age
0,00b74e27ad1cbb2ded8e907fcc49eaaf,6839,477,739,133,684,8.56,10.24,3,0.081134,226,30.58,66,3408,3360,3261,23341,24229,1976-01-01,21,35,Trondheim,Norway,2016-01-01,Has high school and/or college education,M,F,False,False,unknown,40
1,024610702baf540af5637873cd1534e9,26280,1197,464,36,622,4.50,3.97,2,0.000069,56,12.07,16,888,2088,1509,45068,49559,1997-07-04,18,29,Richmond,VA,2016-07-12,Has high school and/or college education,M,F,False,False,unknown,19
2,0a5e3dd8489fe67485ddb7d6adb26ebd,3196,249,303,33,295,4.25,3.47,2,0.033715,106,34.98,17,548,1291,1152,6679,5500,1998-02-06,20,24,unknown,unknown,2019-07-01,Has no high school or college education,M,F,False,False,unknown,21
3,048dd37565ad9cbc24c163ffedffbf58,2077,158,47,28,130,7.89,5.88,6,0.643727,3,6.38,0,94,371,224,6724,7899,1998-02-24,20,25,Edmonton,Alberta,2019-09-25,Has no high school or college education,M,F,False,False,unknown,21
4,0eb998fdde77f9c123c07eace18a5cc1,11946,715,809,444,198,6.82,1.92,3,0.091771,296,36.59,13,1905,5527,6052,60169,96673,1996-11-10,18,27,unknown,unknown,2017-11-17,Has no high school or college education,M,F,True,False,Research Assistant,21


In [27]:
df.shape

(1209, 31)

In [28]:
#df.to_csv('Tinder_Data_v3_Clean_Edition.csv', index=False)

In [29]:
unique_values = df['user_age'].unique()

# Print the unique values
print(unique_values)

[ 40  19  21  25  18  24  36  28  20  29  22  33  30  23  31  26  17  32
  27 106  34  37  15  45  16  39  35  14  38  42 137  41  43  44   0  48
  51 120  47]


In [31]:
unique_values = df['country'].unique()

# Print the unique values
print(unique_values)

['Norway' 'VA' 'unknown' 'Alberta' 'CDMX' 'USA' 'Flanders' 'NSW' 'CA' 'IN'
 'France' 'England' 'Finland' 'Brasil' 'Schleswig-Holstein' 'VIC' 'Texas'
 'Ireland' 'Denmark' 'Arkansas' 'Danmark' 'AB' 'Washington' 'Germany'
 'Bayern' 'Switzerland' 'South Korea' 'ON' 'FL' 'New Zealand' 'OH'
 'Magyarország' 'Deutschland' 'Virginia' 'Hong Kong SAR, China' 'Czechia'
 'United Kingdom' 'Schweiz' 'Yucatán' 'Gujarat' 'California' 'BC'
 'Wisconsin' 'PA' 'New York' 'Nordrhein-Westfalen' 'Scotland' 'MA'
 'Friuli-Venezia Giulia' 'Italia' 'Australia' 'Guatemala' 'Florida'
 'Pennsylvania' 'Baden-Württemberg' 'QC' 'Nederland' 'IL' 'WA'
 'United States' 'Georgia' 'NY' 'Polonya' 'Hungary' 'Lituanie' 'RI'
 'Russia' 'Austria' 'MD' 'Poland' 'Suède' 'Polska' 'Zagreb' 'Indiana'
 'Italy' 'North Holland' 'Illinois' 'Sweden' 'Suomi' 'Norge' 'Sverige'
 'ME' 'MO' 'Sachsen' 'Arizona' 'Uusimaa' 'Canada' 'Groningen' 'CT'
 'Slovakia' 'British Columbia' 'Belgique' 'Aargau' 'OR' 'Romania' 'Česko'
 'Rusya' 'North Rhine-West

In [32]:
# Basic mapping of city/state to country
city_country_mapping = {
    'Trondheim': 'Norway',
    'Richmond': 'USA',
    'Edmonton': 'Canada',
    'Alberta': 'Canada',
    'CDMX': 'Mexico',
    'Flanders': 'Belgium',
    'NSW': 'Australia',
    'CA': 'USA',
    'IN': 'USA',
    'England': 'United Kingdom',
    'VIC': 'Australia',
    'Texas': 'USA',
    'Arkansas': 'USA',
    'Danmark': 'Denmark',
    'AB': 'Canada',
    'Washington': 'USA',
    'Bayern': 'Germany',
    'South Korea': 'South Korea',  # This is already a country but included for clarity
    'ON': 'Canada',
    'FL': 'USA',
    'OH': 'USA',
    'Deutschland': 'Germany',
    'Virginia': 'USA',
    'QC': 'Canada',
    'IL': 'USA',
    'WA': 'USA',
    'United States': 'USA',  # This is already a country but included for clarity
    'Georgia': 'USA',
    'NY': 'USA',
    'RI': 'USA',
    'Austria': 'Austria',  # This is already a country but included for clarity
    'MD': 'USA',
    'Indiana': 'USA',
    'Illinois': 'USA',
    'Sweden': 'Sweden',  # This is already a country but included for clarity
    'ME': 'USA',
    'MO': 'USA',
    'Arizona': 'USA',
    'Uusimaa': 'Finland',
    'Canada': 'Canada',  # This is already a country but included for clarity
    'CT': 'USA',
    'BC': 'Canada',
    'OR': 'USA',
    'WI': 'USA',
    'Stockholm': 'Sweden',
    'DC': 'USA',
    'Zürich': 'Switzerland',
    'NC': 'USA',
    'MI': 'USA',
    'Oregon': 'USA',
    'MT': 'USA',
    'Missouri': 'USA',
    'TN': 'USA',
    'CO': 'USA',
    'PA': 'USA',
    'TX': 'USA',
    'GA': 'USA',
    'NJ': 'USA',
    'MS': 'USA',
    'VT': 'USA',
    'OK': 'USA',
    'Ontario': 'Canada',
    'Tyrol': 'Austria',
    'New Jersey': 'USA',
    'OK': 'USA',
    'PR': 'USA',
    'Maryland': 'USA',
    'LA': 'USA',
    'Berlin': 'Germany',
    'Hawaii': 'USA',
    'Россия': 'Russia'
    # ... additional mappings can be added as needed
}

# Replace city names with country names in the 'country' column using the mapping
df['country'] = df['country'].replace(city_country_mapping)

# Display the unique values in the 'country' column after replacement
unique_countries_after_replacement = df['country'].unique()

unique_countries_after_replacement


array(['Norway', 'VA', 'unknown', 'Canada', 'Mexico', 'USA', 'Belgium',
       'Australia', 'France', 'United Kingdom', 'Finland', 'Brasil',
       'Schleswig-Holstein', 'Ireland', 'Denmark', 'Germany',
       'Switzerland', 'South Korea', 'New Zealand', 'Magyarország',
       'Hong Kong SAR, China', 'Czechia', 'Schweiz', 'Yucatán', 'Gujarat',
       'California', 'Wisconsin', 'New York', 'Nordrhein-Westfalen',
       'Scotland', 'MA', 'Friuli-Venezia Giulia', 'Italia', 'Guatemala',
       'Florida', 'Pennsylvania', 'Baden-Württemberg', 'Nederland',
       'Polonya', 'Hungary', 'Lituanie', 'Russia', 'Austria', 'Poland',
       'Suède', 'Polska', 'Zagreb', 'Italy', 'North Holland', 'Sweden',
       'Suomi', 'Norge', 'Sverige', 'Sachsen', 'Groningen', 'Slovakia',
       'British Columbia', 'Belgique', 'Aargau', 'Romania', 'Česko',
       'Rusya', 'North Rhine-Westphalia', 'Gelderland', 'Israel',
       'Minas Gerais', 'Auvergne-Rhône-Alpes', 'Capital Region',
       'Catalunya', 'Эстония

In [35]:
df['country'] = df['country'].replace(city_country_mapping)


In [42]:
# Extend the mapping with more city/state to country mappings
extended_mapping = {
    'VA': 'USA',  # Virginia
    'CDMX': 'Mexico',  # Mexico City
    'NSW': 'Australia',  # New South Wales
    'CA': 'USA',  # California
    'IN': 'USA',  # Indiana
    'VIC': 'Australia',  # Victoria
    'AB': 'Canada',  # Alberta
    'Bayern': 'Germany',  # Bavaria
    'ON': 'Canada',  # Ontario
    'FL': 'USA',  # Florida
    'OH': 'USA',  # Ohio
    'Deutschland': 'Germany',
    'QC': 'Canada',  # Quebec
    'IL': 'USA',  # Illinois
    'WA': 'USA',  # Washington
    'GA': 'USA',  # Georgia
    'NY': 'USA',  # New York
    'RI': 'USA',  # Rhode Island
    'MD': 'USA',  # Maryland
    'ME': 'USA',  # Maine
    'MO': 'USA',  # Missouri
    'AZ': 'USA',  # Arizona
    'CT': 'USA',  # Connecticut
    'BC': 'Canada',  # British Columbia
    'WI': 'USA',  # Wisconsin
    'SD': 'USA',  # South Dakota
    'MI': 'USA',  # Michigan
    'MT': 'USA',  # Montana
    'TN': 'USA',  # Tennessee
    'CO': 'USA',  # Colorado
    'TX': 'USA',  # Texas
    'NJ': 'USA',  # New Jersey
    'MS': 'USA',  # Mississippi
    'VT': 'USA',  # Vermont
    'OK': 'USA',  # Oklahoma
    'PR': 'USA',  # Puerto Rico
    'LA': 'USA',  # Louisiana
    'HI': 'USA',  # Hawaii
    'PA': 'USA',  # Pennsylvania
    'Россия': 'Russia',
    'Brasil': 'Brazil',
    'Schweiz': 'Switzerland',
    'Magyarország': 'Hungary',
    'España': 'Spain',
    'Nederland': 'Netherlands',
    'Sverige': 'Sweden',
    'Suomi': 'Finland',
    'Norge': 'Norway',
    'Danmark': 'Denmark',
    'Österreich': 'Austria',
    'Italia': 'Italy',
    'Polska': 'Poland',
    'Belgique': 'Belgium',
    'België': 'Belgium',
    'Česko': 'Czechia',
    'Česká republika': 'Czechia',
    'Slovensko': 'Slovakia',
    'Türkiye': 'Turkey',
    'Беларусь': 'Belarus',
    'Σουηδία': 'Sweden',
    'Эстония': 'Estonia',
    'Украина': 'Ukraine',
    'Francia': 'France',
    'Pernambuco': 'Brazil',
    'Rheinland-Pfalz': 'Germany',
    'Niedersachsen': 'Germany',
    'Hessen': 'Germany',
    'Baden-Württemberg': 'Germany',
    'Nordrhein-Westfalen': 'Germany',
    'Schleswig-Holstein': 'Germany',
    'Reino Unido': 'United Kingdom',
    'Rusya': 'Russia',
    'Polonia': 'Poland',
    'Polonya': 'Poland',
    'Andhra Pradesh': 'India',
    'Mississippi': 'USA',
    'Yucatán': 'Mexico',
    'Gujarat': 'India',
    'Zagreb': 'Croatia',
    '北京市': 'China',
    'Jalisco': 'Mexico',
    'Réunion': 'France',
    'Hallands län': 'Sweden',
    'Mazowieckie': 'Poland',
    'Małopolskie': 'Poland',
    'Jönköpings län': 'Sweden',
    'Kujawsko-Pomorskie': 'Poland',
    'Dolnośląskie': 'Poland',
    'Montevideo Department': 'Uruguay',
    'Nariño': 'Colombia',
    'Jilin': 'China',
    'Lazio': 'Italy',
    'Piemonte': 'Italy'
    # ... additional mappings can be added as needed
}

# Merge the new mapping with the existing mapping
city_country_mapping.update(extended_mapping)

# Replace city and state names with country names in the 'country' column using the updated mapping
df['country'] = df['country'].replace(city_country_mapping)

# Display the unique values in the 'country' column after the extended replacement
unique_countries_after_extended_replacement = df['country'].unique()

unique_countries_after_extended_replacement


array(['Norway', 'USA', 'unknown', 'Canada', 'Mexico', 'Belgium',
       'Australia', 'France', 'United Kingdom', 'Finland', 'Brazil',
       'Germany', 'Ireland', 'Denmark', 'Switzerland', 'South Korea',
       'New Zealand', 'Hungary', 'Hong Kong SAR, China', 'Czechia',
       'India', 'California', 'Wisconsin', 'New York', 'Scotland', 'MA',
       'Friuli-Venezia Giulia', 'Italy', 'Guatemala', 'Florida',
       'Pennsylvania', 'Netherlands', 'Poland', 'Lituanie', 'Russia',
       'Austria', 'Suède', 'Croatia', 'North Holland', 'Sweden',
       'Sachsen', 'Groningen', 'Slovakia', 'British Columbia', 'Aargau',
       'Romania', 'North Rhine-Westphalia', 'Gelderland', 'Israel',
       'Minas Gerais', 'Auvergne-Rhône-Alpes', 'Capital Region',
       'Catalunya', 'Estonia', 'Michigan', 'South Dakota', 'Vaud', 'Ohio',
       'North Carolina', 'NL', 'Spain', 'China', 'Łódzkie', 'País Vasco',
       'Île-de-France', 'Western Australia', 'Philippines',
       'United Arab Emirates', 'Județul

In [52]:
# Further mapping for potential cities/states to countries
additional_mapping_2 = {
    'Hong Kong SAR, China': 'China',
    'California': 'USA',
    'Wisconsin': 'USA',
    'New York': 'USA',
    'Scotland': 'United Kingdom',
    'Friuli-Venezia Giulia': 'Italy',
    'Florida': 'USA',
    'Pennsylvania': 'USA',
    'Lituanie': 'Lithuania',
    'Suède': 'Sweden',
    'North Holland': 'Netherlands',
    'Sachsen': 'Germany',
    'Groningen': 'Netherlands',
    'British Columbia': 'Canada',
    'Aargau': 'Switzerland',
    'North Rhine-Westphalia': 'Germany',
    'Gelderland': 'Netherlands',
    'Minas Gerais': 'Brazil',
    'Auvergne-Rhône-Alpes': 'France',
    'Capital Region': 'Denmark',  # Assuming it's the Capital Region of Denmark
    'Catalunya': 'Spain',
    'Michigan': 'USA',
    'South Dakota': 'USA',
    'Vaud': 'Switzerland',
    'Ohio': 'USA',
    'North Carolina': 'USA',
    'Łódzkie': 'Poland',
    'País Vasco': 'Spain',
    'Île-de-France': 'France',
    'Western Australia': 'Australia',
    'Județul Sibiu': 'Romania',
    'County Dublin': 'Ireland',
    'Rio Grande do Sul': 'Brazil',
    'Nevada': 'USA',
    'Australian Capital Territory': 'Australia',
    'Veneto': 'Italy',
    'מחוז תל אביב': 'Israel',
    'County Cork': 'Ireland',
    'Rogaland': 'Norway',
    'Canarias': 'Spain',
    'Federal Territory of Kuala Lumpur': 'Malaysia',
    'Community of Madrid': 'Spain',
    'Québec': 'Canada',
    'Alabama': 'USA',
    'Victoria': 'Australia',
    'Lombardy': 'Italy',
    'Comunidad de Madrid': 'Spain',
    'Styria': 'Austria',
    'Seoul': 'South Korea',
    'Incheon': 'South Korea',
    'Cluj County': 'Romania',
    'Hovedstaden': 'Denmark',
    'Oost-Vlaanderen': 'Belgium',
    'Utah': 'USA',
    'New South Wales': 'Australia',
    "Provence-Alpes-Côte d'Azur": 'France',
    'District of Columbia': 'USA',
    'New Hampshire': 'USA',
    'QLD': 'Australia',
    'Zuid-Holland': 'Netherlands',
    'Noord-Holland': 'Netherlands',
    'Județul Iași': 'Romania',
    'Nordjylland': 'Denmark',
    'Oklahoma': 'USA',
    'Silesian Voivodeship': 'Poland',
    'Rio de Janeiro': 'Brazil',
    'Santa Catarina': 'Brazil',
    'Hlavní město Praha': 'Czechia',
    'Grand Est': 'France',
    'Alicante': 'Spain',
    'Iași County': 'Romania',
    'Athens': 'Greece',
    'MA': 'USA',
    'KS':'USA',
    'NL': 'Mexico',
    'asdf':'unknown'
}

# Merge the new mapping with the existing mapping
city_country_mapping.update(additional_mapping_2)

# Replace city and state names with country names in the 'country' column using the updated mapping
df['country'] = df['country'].replace(city_country_mapping)

# Display the unique values in the 'country' column after the additional replacement
unique_countries_after_additional_replacement = df['country'].unique()

unique_countries_after_additional_replacement


array(['Norway', 'USA', 'unknown', 'Canada', 'Mexico', 'Belgium',
       'Australia', 'France', 'United Kingdom', 'Finland', 'Brazil',
       'Germany', 'Ireland', 'Denmark', 'Switzerland', 'South Korea',
       'New Zealand', 'Hungary', 'China', 'Czechia', 'India', 'Italy',
       'Guatemala', 'Netherlands', 'Poland', 'Lithuania', 'Russia',
       'Austria', 'Sweden', 'Croatia', 'Slovakia', 'Romania', 'Israel',
       'Spain', 'Estonia', 'Philippines', 'United Arab Emirates',
       'Belarus', 'Turkey', 'Argentina', 'Portugal', 'Iceland', 'Taiwan',
       'Malaysia', 'Luxembourg', 'Thailand', 'Colombia', 'Greece',
       'Ukraine', 'Japan', 'Morocco', 'Suisse', 'Bulgaria', 'Singapore',
       'Uruguay', 'Latvia', 'South Africa'], dtype=object)

In [54]:
#filtered_data = df[df['country'].isin(['unknown'])]

#filtered_data.head()

In [55]:
df.head()

,_id,sum_app_opens,no_of_days,nrOfConversations,longestConversation,longestConversationInDays,averageConversationLength,averageConversationLengthInDays,medianConversationLength,medianConversationLengthInDays,nrOfOneMessageConversations,percentOfOneMessageConversations,nrOfGhostingsAfterInitialMessage,no_of_matches,no_of_msgs_sent,no_of_msgs_received,swipe_likes,swipe_passes,birthDate,ageFilterMin,ageFilterMax,cityName,country,createDate,education,gender,interestedIn,instagram,spotify,jobTitle,user_age
0,00b74e27ad1cbb2ded8e907fcc49eaaf,6839,477,739,133,684,8.56,10.24,3,0.081134,226,30.58,66,3408,3360,3261,23341,24229,1976-01-01,21,35,Trondheim,Norway,2016-01-01,Has high school and/or college education,M,F,False,False,unknown,40
1,024610702baf540af5637873cd1534e9,26280,1197,464,36,622,4.50,3.97,2,0.000069,56,12.07,16,888,2088,1509,45068,49559,1997-07-04,18,29,Richmond,USA,2016-07-12,Has high school and/or college education,M,F,False,False,unknown,19
2,0a5e3dd8489fe67485ddb7d6adb26ebd,3196,249,303,33,295,4.25,3.47,2,0.033715,106,34.98,17,548,1291,1152,6679,5500,1998-02-06,20,24,unknown,unknown,2019-07-01,Has no high school or college education,M,F,False,False,unknown,21
3,048dd37565ad9cbc24c163ffedffbf58,2077,158,47,28,130,7.89,5.88,6,0.643727,3,6.38,0,94,371,224,6724,7899,1998-02-24,20,25,Edmonton,Canada,2019-09-25,Has no high school or college education,M,F,False,False,unknown,21
4,0eb998fdde77f9c123c07eace18a5cc1,11946,715,809,444,198,6.82,1.92,3,0.091771,296,36.59,13,1905,5527,6052,60169,96673,1996-11-10,18,27,unknown,unknown,2017-11-17,Has no high school or college education,M,F,True,False,Research Assistant,21


In [56]:
unique_values = df['cityName'].unique()

# Print the unique values
print(unique_values)

['Trondheim' 'Richmond' 'unknown' 'Edmonton' 'Ciudad de México' 'Boston'
 'Sint-Martens-Latem' 'Cabramatta' 'Newport Beach' 'Indianapolis' 'Oslo'
 'Paris' 'Lyon' 'London' 'San Diego' 'Cherbourg-en-Cotentin' 'Jyväskylä'
 'São Paulo' 'Santa Clara' 'Kiel' 'Carlton' 'Dublin' 'Skive' 'Batesville'
 'Vienna' 'Garden Grove' 'asdf' 'Seattle' 'Munich' 'München'
 'Saint-Maur-des-Fossés' 'Basel' 'Seongnam-si' 'Toronto' 'Pompano Beach'
 'Wellington' 'Columbus' 'Debrecen' 'Mannheim' 'Roanoke' 'Central'
 'Prague' 'Swindon' 'Zürich' 'Mérida' 'Odense' 'Ahmedabad' 'Gofuckurself'
 'Pasadena' 'Vancouver' 'Marseille' 'San Francisco' 'Oshkosh' 'Glen Mills'
 'Langley' 'Rochester' 'Bad Salzuflen' 'Altoona' 'Edinburgh' 'Manchester'
 'Somerville' 'København' 'Udine' 'Melbourne' 'Fort Lauderdale' 'Campbell'
 'Los Angeles' 'Roma' 'Gold Coast' 'Guatemala' 'Merritt Island' 'Malvern'
 'Leonberg' 'Oakland' 'Montréal' 'Alkmaar' 'Mississauga' 'Chicago'
 'Mount Vernon' 'Medford' 'Athens' 'Limerick' 'Sheffield' 'Budapest

In [73]:
# Mapping for cities written in a different language to their English equivalents
city_translation_mapping = {
    'Ciudad de México': 'Mexico City',
    'München': 'Munich',
    'København': 'Copenhagen',
    '北京市': 'Beijing',
    'Αθήνα': 'Athens',
    'Γκέτεμποργκ': 'Gothenburg',
    'Brașov': 'Brasov',
    'Praha': 'Prague',
    'Voronej': 'Voronezh',
    'Минск': 'Minsk',
    'Θεσσαλονίκη': 'Thessaloniki',
    'Киев': 'Kiev',
    'Kyiv': 'Kiev',
    'Київ': 'Kiev',
    'Varsovia': 'Warsaw',
    'Tel Aviv-Yafo': 'Tel Aviv',
    'Genève': 'Geneva',
    'Estrasburgo': 'Strasbourg',
    'Bruxelles': 'Brussels',
    '多伦多': 'Toronto',
    'Москва': 'Moscow',
    'São Paulo': 'São Paulo',
    'Roma': 'Rome',
    'Montréal': 'Montreal',
    'Warszawa': 'Warsaw',
    'Milano': 'Milan',
    'Bruxelles': 'Brussels',
    'Genève': 'Geneva',
    'Минск': 'Minsk',
    'Киев': 'Kiev',
    'Θεσσαλονίκη': 'Thessaloniki',
    'Київ': 'Kiev',
    'Москва': 'Moscow',
    'Roma': 'Rome',
    'Warszawa': 'Warsaw',
    'Göteborg': 'Gothenburg',
    'Минск': 'Minsk',
    'Γκέτεμποργκ': 'Gothenburg',
    'Тель-Авив': 'Tel Aviv',
    'Яффа': 'Jaffa',
    'Київ': 'Kiev',
    '北京市': 'Beijing',
    'Łódź': 'Lodz',
    'τη Θεσσαλονίκη': 'Thessaloniki',
    '北京市': 'Beijing',
    '西雅图': 'Seattle',
    'Гётеборг': 'Gothenburg',
    'Warszawa': 'Warsaw',
    'Кёльн': 'Cologne',
    'Мюнхен': 'Munich',
    'Париж': 'Paris',
    'Стокгольм': 'Stockholm',
    'Вена': 'Vienna',
    'Барселона': 'Barcelona',
    'Торонто': 'Toronto',
    'Эдинбург': 'Edinburgh',
    'Гамбург': 'Hamburg',
    'Лос-Анджелес': 'Los Angeles',
    'Милан': 'Milan',
    'Brno': 'Brünn',
    '東京': 'Tokyo',
    'Αθήνα': 'Athens',
    'Rīga': 'Riga',
    'Минск': 'Minsk',
    'תל אביב-יפו': 'Tel Aviv',
    'Мюнхен': 'Munich',
    'Париж': 'Paris',
    'Стокгольм': 'Stockholm',
    'Вена': 'Vienna',
    'Барселона': 'Barcelona',
    'Торонто': 'Toronto',
    'Эдинбург': 'Edinburgh',
    'Гамбург': 'Hamburg',
    'Лос-Анджелес': 'Los Angeles',
    'Милан': 'Milan',
    'Göteborg': 'Gothenburg',
    'Москва': 'Moscow',
    'Киев': 'Kiev',
    'Warszawa': 'Warsaw',
    'Кёльн': 'Cologne',
    'Мюнхен': 'Munich',
    'Париж': 'Paris',
    'Стокгольм': 'Stockholm',
    'Вена': 'Vienna',
    'Барселона': 'Barcelona',
    'Торонто': 'Toronto',
    'Эдинбург': 'Edinburgh',
    'Гамбург': 'Hamburg',
    'Лос-Анджелес': 'Los Angeles',
    'Милан': 'Milan',
    'Brünn': 'Brno',
    'Stockholm County': 'Stockholm',
    'England (Not a City)': 'England',
    'Bruxelles': 'Brussels',
    'Brüssel': 'Brussels',
    'Genève': 'Geneva',
    'Göteborg': 'Gothenburg',
    'Köln': 'Cologne',
    'München': 'Munich',
    'Praha': 'Prague',
    'Roma': 'Rome',
    'Αθήνα': 'Athens',
    'Київ': 'Kiev',
    'Москва': 'Moscow',
    'תל אביב': 'Tel Aviv',
    'Wien': 'Vienna',
    'København': 'Copenhagen',
    'Milano': 'Milan',
    'Минск': 'Minsk',
    'Beograd': 'Belgrade',
    'Белград': 'Belgrade',
    'Θεσσαλονίκη': 'Thessaloniki',
    'Βερολίνο': 'Berlin',
    'Βιέννη': 'Vienna',
    'Берлин': 'Berlin',
    'София': 'Sofia',
    'Лондон': 'London',
    'Будапешт': 'Budapest',
    'Амстердам': 'Amsterdam',
    'Глазго': 'Glasgow',
    'Милано': 'Milan',
    'Рим': 'Rome',
    'Βρυξέλλες': 'Brussels',
    'Μάντσεστερ': 'Manchester',
    'Казань': 'Kazan',
    'Λονδίνο': 'London',
    'Μπριστόλ': 'Bristol',
    'Λίβερπουλ': 'Liverpool',
    'Κοπεγχάγη': 'Copenhagen',
    'Μιλάνο': 'Milan',
    'Ρώμη': 'Rome',
    'Глазго': 'Glasgow',
    'Санкт-Петербург': 'Saint Petersburg',
    'Львов': 'Lviv',
    'Марсель': 'Marseille',
    'Цюрих': 'Zurich',
    'Манчестер': 'Manchester',
    'Брюссель': 'Brussels',
    'Лиссабон': 'Lisbon',
    'Антверпен': 'Antwerp',
    'Гент': 'Ghent',
    'Севилья': 'Seville',
    'Барселона': 'Barcelona',
    'Цинциннати': 'Cincinnati',
    'Милуоки': 'Milwaukee',
    'Вашингтон': 'Washington',
    'Сан-Франциско': 'San Francisco',
    'Бостон': 'Boston',
    'Дублин': 'Dublin',
    'Мадрид': 'Madrid',
    'Рима': 'Rome',
    'Бристоль': 'Bristol',
    'Мадрид': 'Madrid',
    'Белфаст': 'Belfast',
    'Глазго': 'Glasgow',
    'Дублин': 'Dublin',
    'Ливерпуль': 'Liverpool',
    'Лондон': 'London',
    'Манчестер': 'Manchester',
    'Эдинбург': 'Edinburgh',
    'Кардифф': 'Cardiff',
    'Бирмингем': 'Birmingham',
    'Бирмингем': 'Birmingham',
    'Кардифф': 'Cardiff',
    'Ливерпуль': 'Liverpool',
    'Лондон': 'London',
    'Манчестер': 'Manchester',
    'Шеффилд': 'Sheffield',
    'Эдинбург': 'Edinburgh',
    'Глазго': 'Glasgow',
    'Лидс': 'Leeds',
    'Ньюкасл': 'Newcastle'
}

# Apply the translation mapping to the 'cityName' column
df['cityName'] = df['cityName'].replace(city_translation_mapping)

# Display the unique values in the 'cityName' column after the translation
unique_city_names_after_translation = df['cityName'].unique()

unique_city_names_after_translation


array(['Trondheim', 'Richmond', 'unknown', 'Edmonton', 'Mexico City',
       'Boston', 'Sint-Martens-Latem', 'Cabramatta', 'Newport Beach',
       'Indianapolis', 'Oslo', 'Paris', 'Lyon', 'London', 'San Diego',
       'Cherbourg-en-Cotentin', 'Jyväskylä', 'São Paulo', 'Santa Clara',
       'Kiel', 'Carlton', 'Dublin', 'Skive', 'Batesville', 'Vienna',
       'Garden Grove', 'asdf', 'Seattle', 'Munich',
       'Saint-Maur-des-Fossés', 'Basel', 'Seongnam-si', 'Toronto',
       'Pompano Beach', 'Wellington', 'Columbus', 'Debrecen', 'Mannheim',
       'Roanoke', 'Central', 'Prague', 'Swindon', 'Zürich', 'Mérida',
       'Odense', 'Ahmedabad', 'Gofuckurself', 'Pasadena', 'Vancouver',
       'Marseille', 'San Francisco', 'Oshkosh', 'Glen Mills', 'Langley',
       'Rochester', 'Bad Salzuflen', 'Altoona', 'Edinburgh', 'Manchester',
       'Somerville', 'Copenhagen', 'Udine', 'Melbourne',
       'Fort Lauderdale', 'Campbell', 'Los Angeles', 'Rome', 'Gold Coast',
       'Guatemala', 'Merritt Isla

In [83]:
#df.to_csv('Tinder_Data_v3_Clean_Edition.csv', index=False)

In [75]:
df.head()

,_id,sum_app_opens,no_of_days,nrOfConversations,longestConversation,longestConversationInDays,averageConversationLength,averageConversationLengthInDays,medianConversationLength,medianConversationLengthInDays,nrOfOneMessageConversations,percentOfOneMessageConversations,nrOfGhostingsAfterInitialMessage,no_of_matches,no_of_msgs_sent,no_of_msgs_received,swipe_likes,swipe_passes,birthDate,ageFilterMin,ageFilterMax,cityName,country,createDate,education,gender,interestedIn,instagram,spotify,jobTitle,user_age
0,00b74e27ad1cbb2ded8e907fcc49eaaf,6839,477,739,133,684,8.56,10.24,3,0.081134,226,30.58,66,3408,3360,3261,23341,24229,1976-01-01,21,35,Trondheim,Norway,2016-01-01,Has high school and/or college education,M,F,False,False,unknown,40
1,024610702baf540af5637873cd1534e9,26280,1197,464,36,622,4.50,3.97,2,0.000069,56,12.07,16,888,2088,1509,45068,49559,1997-07-04,18,29,Richmond,USA,2016-07-12,Has high school and/or college education,M,F,False,False,unknown,19
2,0a5e3dd8489fe67485ddb7d6adb26ebd,3196,249,303,33,295,4.25,3.47,2,0.033715,106,34.98,17,548,1291,1152,6679,5500,1998-02-06,20,24,unknown,unknown,2019-07-01,Has no high school or college education,M,F,False,False,unknown,21
3,048dd37565ad9cbc24c163ffedffbf58,2077,158,47,28,130,7.89,5.88,6,0.643727,3,6.38,0,94,371,224,6724,7899,1998-02-24,20,25,Edmonton,Canada,2019-09-25,Has no high school or college education,M,F,False,False,unknown,21
4,0eb998fdde77f9c123c07eace18a5cc1,11946,715,809,444,198,6.82,1.92,3,0.091771,296,36.59,13,1905,5527,6052,60169,96673,1996-11-10,18,27,unknown,unknown,2017-11-17,Has no high school or college education,M,F,True,False,Research Assistant,21


In [76]:
df.shape

(1209, 31)

In [82]:
unique_values = df['spotify'].unique()

# Print the unique values
print(unique_values)

[False  True]


In [81]:
df['spotify'] = df['spotify'].fillna(False)


In [84]:
unique_values = df['jobTitle'].unique()

# Print the unique values
print(unique_values)

['unknown' 'Research Assistant' 'False' 'Physician' 'Scientist'
 'Product designer' 'Sr. Director' 'Architecte IT' 'Software Developer'
 'Web Developer' 'Barista' 'Student' 'Analyst' 'Medical Care Technician'
 'Chemist' 'CEO' 'Software engineer' 'Technology Analyst'
 'Brow Waxing Expert' 'Développeur Web' 'Software Engineer'
 'Property Manager' 'Content marketing consultant' 'Software developer'
 'Writer' 'Socio' 'Manager' 'Ingénieur'
 'Engineer, Axe throwing instructor' 'Sales' 'Admin'
 'Industrial Engineering Student' 'Anlagenmechaniker' 'Grad Student'
 'App Developer' 'concept artist' 'Paralegal' 'Therapist' 'Teacher'
 'Marketing' 'Civil Engineer' 'Truss Designer' 'Software Engineer Intern'
 'Fusion Engineer' 'Systemutvikler' 'Mechanical Engineer'
 'Graphiste Motion Designer' 'Engineer'
 'Marketing & Business Development Manager' 'Редактор.' 'Consultant'
 'Energy Trader' 'Server' 'Pfizer Vaccine Distributor' 'Med Student'
 'Environmental Science student' 'World Class Nap Taker' 'Ent

In [93]:
# Manually translating the recognized non-English job titles
translation_map = {
    'Développeur Web': 'Web Developer',
    'Ingénieur': 'Engineer',
    'Редактор': 'Editor',
    'Sivilingeniør': 'Civil Engineer',
    'Chargé de traitement statistique': 'Statistic Processing Officer',
    'Geschäftsführung': 'Management',
    'Öffentlicher Dienst': 'Public Service',
    'Étudiant': 'Student',
    'Ohjelmistokehittäjä': 'Software Developer',
    'Fachinformatiker für Systemintegration': 'IT Specialist for System Integration',
    'Yazılım Mühendisi': 'Software Engineer',
    'Civilingenjör': 'Civil Engineer',
    'Egenföretagare': 'Entrepreneur',
    'Designer Gráfico': 'Graphic Designer',
    'Dev Front-End, futuro Psicólogo': 'Front-End Developer, Future Psychologist',
    'tillfälligt i produktion': 'Temporarily in Production',
    'Boss par intérim': 'Temporary Boss',
    'Vagt på Kongelige Slotte og Palæer': 'Guard at the Royal Castles and Palaces',
    'я самогонщик': 'Moonshiner',
    'Analista de sistemas de automação': 'Automation Systems Analyst',
    'Редактор.': 'Editor',
    '打工仔': 'Worker (Wage Earner)',
    'Socio': 'Partner',
    'Anlagenmechaniker': 'Plant Mechanic',
    'Systemutvikler': 'System Developer',
    'Rennmechaniker': 'Racing Mechanic',
    'Video Ingenieur': 'Video Engineer',
    'Apputvecklare': 'App Developer',
    'Farmaceutický analytik': 'Pharmaceutical Analyst',
    'Studente Ingegneria Biomedica': 'Biomedical Engineering Student',
    '30 euro vom flaschensammeln entfernt': '30 euros away from collecting bottles',
    'בצבא': 'In the army',
    'Astronaute': 'Astronaut',
    'Jurist': 'Lawyer',
    'Informatikstudent': 'Computer Science Student',
    'Opiskelija, historia': 'History Student',
    'Masterstudent Informatik': "Master's Student in Computer Science",
    'Ho A Che Fare Con I Turisti.': 'I deal with tourists',
    'Kunst underviser': 'Art Teacher',
    'Ingenieur Apprenti': 'Apprentice Engineer',
    'Majitel webu': 'Website Owner',
    'Pengar': 'Money',
    'Empreendedor': 'Entrepreneur',
    'Tjener': 'Waiter',
    'Graduando': 'Graduating Student',
    'Was mit Computern': 'Something with computers'
    
}

# Replace non-English job titles with their translations in the dataset
df['jobTitle'] = df['jobTitle'].replace(translation_map)

# Display the updated job titles for verification
df['jobTitle'].unique()


array(['unknown', 'Research Assistant', 'False', 'Physician', 'Scientist',
       'Product designer', 'Sr. Director', 'Architecte IT',
       'Software Developer', 'Web Developer', 'Barista', 'Student',
       'Analyst', 'Medical Care Technician', 'Chemist', 'CEO',
       'Software engineer', 'Technology Analyst', 'Brow Waxing Expert',
       'Software Engineer', 'Property Manager',
       'Content marketing consultant', 'Software developer', 'Writer',
       'Partner', 'Manager', 'Engineer',
       'Engineer, Axe throwing instructor', 'Sales', 'Admin',
       'Industrial Engineering Student', 'Plant Mechanic', 'Grad Student',
       'App Developer', 'concept artist', 'Paralegal', 'Therapist',
       'Teacher', 'Marketing', 'Civil Engineer', 'Truss Designer',
       'Software Engineer Intern', 'Fusion Engineer', 'System Developer',
       'Mechanical Engineer', 'Graphiste Motion Designer',
       'Marketing & Business Development Manager', 'Editor', 'Consultant',
       'Energy Trader'

In [94]:
df.to_csv('Tinder_Data_v3_Clean_Edition.csv', index=False)

In [95]:
df.head()

,_id,sum_app_opens,no_of_days,nrOfConversations,longestConversation,longestConversationInDays,averageConversationLength,averageConversationLengthInDays,medianConversationLength,medianConversationLengthInDays,nrOfOneMessageConversations,percentOfOneMessageConversations,nrOfGhostingsAfterInitialMessage,no_of_matches,no_of_msgs_sent,no_of_msgs_received,swipe_likes,swipe_passes,birthDate,ageFilterMin,ageFilterMax,cityName,country,createDate,education,gender,interestedIn,instagram,spotify,jobTitle,user_age
0,00b74e27ad1cbb2ded8e907fcc49eaaf,6839,477,739,133,684,8.56,10.24,3,0.081134,226,30.58,66,3408,3360,3261,23341,24229,1976-01-01,21,35,Trondheim,Norway,2016-01-01,Has high school and/or college education,M,F,False,False,unknown,40
1,024610702baf540af5637873cd1534e9,26280,1197,464,36,622,4.50,3.97,2,0.000069,56,12.07,16,888,2088,1509,45068,49559,1997-07-04,18,29,Richmond,USA,2016-07-12,Has high school and/or college education,M,F,False,False,unknown,19
2,0a5e3dd8489fe67485ddb7d6adb26ebd,3196,249,303,33,295,4.25,3.47,2,0.033715,106,34.98,17,548,1291,1152,6679,5500,1998-02-06,20,24,unknown,unknown,2019-07-01,Has no high school or college education,M,F,False,False,unknown,21
3,048dd37565ad9cbc24c163ffedffbf58,2077,158,47,28,130,7.89,5.88,6,0.643727,3,6.38,0,94,371,224,6724,7899,1998-02-24,20,25,Edmonton,Canada,2019-09-25,Has no high school or college education,M,F,False,False,unknown,21
4,0eb998fdde77f9c123c07eace18a5cc1,11946,715,809,444,198,6.82,1.92,3,0.091771,296,36.59,13,1905,5527,6052,60169,96673,1996-11-10,18,27,unknown,unknown,2017-11-17,Has no high school or college education,M,F,True,False,Research Assistant,21


In [96]:
column_names = df.columns.tolist()
print(column_names)


['_id', 'sum_app_opens', 'no_of_days', 'nrOfConversations', 'longestConversation', 'longestConversationInDays', 'averageConversationLength', 'averageConversationLengthInDays', 'medianConversationLength', 'medianConversationLengthInDays', 'nrOfOneMessageConversations', 'percentOfOneMessageConversations', 'nrOfGhostingsAfterInitialMessage', 'no_of_matches', 'no_of_msgs_sent', 'no_of_msgs_received', 'swipe_likes', 'swipe_passes', 'birthDate', 'ageFilterMin', 'ageFilterMax', 'cityName', 'country', 'createDate', 'education', 'gender', 'interestedIn', 'instagram', 'spotify', 'jobTitle', 'user_age']
